# **Ejercicios de pair programming Módulo 2 Sprint 2**
## **ETL: Lección 1**
### **Extracción**

En el marco de un proyecto de análisis de datos, se requiere realizar una ETL para integrar información de distintas fuentes de datos. La empresa para la que trabajamos cuenta con una base de datos de la informacion sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde el año 1998 hasta el 2022. Quieren enriquecer los datos con información de la generación de energías renovables y no renovables y crear una base de datos más completa. Para esto nos piden que usemos la API de REData la cual nos da mucha información sobre el mercado energético en España. La documentación de la API la encontraréis 

1. Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. NOTA: nos piden que el código debe estar en una función. Al final deberemos tener un dataframe como el siguiente:

In [5]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np

In [18]:
df_completo = pd.DataFrame()
for año in range(2011, 2023):

    url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=day"
    response = requests.get(url=url)

    fichero_json_renovable = response.json()["included"][0]["attributes"]["values"] 
    fichero_json_no_renovable = response.json()["included"][1]["attributes"]["values"]

    df_renovables = pd.json_normalize(fichero_json_renovable)
    df_no_renovables = pd.json_normalize(fichero_json_no_renovable)

    df_renovables["tipo"] = response.json()["included"][0]["type"]
    df_no_renovables["tipo"] = response.json()["included"][1]["type"]
    
    df_completo = pd.concat([df_completo, df_renovables, df_no_renovables], axis = 0)

In [25]:
df_completo.sample(2)

,value,percentage,datetime,tipo
332,200689.1655,0.235383,2017-11-29T00:00:00.000+01:00,Renovable
27,426427.5725,0.550743,2021-01-28T00:00:00.000+01:00,Renovable


In [56]:
df_completo.to_csv("energias.csv")

2.  Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [26]:
#Para este segundo ejercicio, en aras de la optimización, decidimos sacar la información por años.

cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [53]:
df_completo_ccaa = pd.DataFrame()
for año in range(2011, 2023):

    for k, v in cod_comunidades.items():
        
        url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
    
        response2 = requests.get(url=url)

        fichero_json_no_renovable = response2.json()["included"][0]["attributes"]["values"] 
        
        df_no_renovables = pd.json_normalize(fichero_json_renovable)
       
        df_no_renovables["tipo"] = response2.json()["included"][0]["type"]


        df_no_renovables["comunidad"] = k

        df_no_renovables["id_comunidad"] = v

    
        df_completo_ccaa = pd.concat([df_completo_ccaa, df_no_renovables], axis = 0)
